In [19]:
import numpy as np
import pandas as pd

In [20]:
file_path = '쓰레기 배출/서울특별시 양천구_구역별쓰레기배출량_20231115.csv'

In [21]:
try:
    df = pd.read_csv(file_path, encoding = 'cp949')
    print("CSV 파일 로드 완료(utf-8)!")
    print(df.head()) # 처음 몇 줄을 출력하여 데이터 확인
except FileNotFoundError:
    print(f"경로 '{file_path}'에 파일이 없습니다. 파일 경로를 확인해주세요.")
except Exception as e:
    print(f"파일을 로드하는 동안 오류가 발생했습니다: {e}")
    

CSV 파일 로드 완료(utf-8)!
   기준년도  기준월   구역                                 행정동  일반생활(톤)  음식물류(톤)  \
0  2019    1  1구역                          신월1·2·3·5동      524      427   
1  2019    1  2구역      신월4·6·7동, 신정3동, 신정1동 1031,1032      591      484   
2  2019    1  3구역     목1동, 신정2동, 신정4동(943~1013, 1302)      552      500   
3  2019    1  4구역          신정1동(1031,1032 제외), 신정6·7동      432      368   
4  2019    1  5구역  목2·3·4동, 신정4동(872~942, 1305, 1308)      601      544   

   재활용품(톤)  
0      NaN  
1      NaN  
2      NaN  
3      NaN  
4      NaN  


In [22]:
# 1. 기준년도를 문자열로 변환
df['기준년도'] = df['기준년도'].astype(str)

# 2. 기준월을 문자열로 변환하고, 한 자리 수 앞에 '0'을 붙임
df['기준월'] = df['기준월'].astype(str).str.zfill(2)

# 3. 새로운 '년월' 컬럼 생성
df['년월'] = df['기준년도'] + '-' + df['기준월']

# 이제 필요없는 기준년도와 기준월 컬럼을 삭제
df = df.drop(columns=['기준년도', '기준월', '행정동'])

# 4. 컬럼 순서 재배치
new_columns = ['년월'] + [col for col in df.columns if col != '년월']
df = df[new_columns]

#출력
print(df)

          년월   구역  일반생활(톤)  음식물류(톤)  재활용품(톤)
0    2019-01  1구역      524      427      NaN
1    2019-01  2구역      591      484      NaN
2    2019-01  3구역      552      500      NaN
3    2019-01  4구역      432      368      NaN
4    2019-01  5구역      601      544      NaN
..       ...  ...      ...      ...      ...
343  2023-10  2구역      672      484    170.0
344  2023-10  3구역      608      441    156.0
345  2023-10  4구역      505      365     43.0
346  2023-10  5구역      689      530    449.0
347  2023-10  6구역      276      223      NaN

[348 rows x 5 columns]


In [24]:
# 1. '일반생활(톤)'과 '음식물류(톤)' 컬럼을 숫자형으로 변환 (오류 방지)
df['일반생활(톤)'] = pd.to_numeric(df['일반생활(톤)'], errors='coerce')
df['음식물류(톤)'] = pd.to_numeric(df['음식물류(톤)'], errors='coerce')

# 2. 두 컬럼의 값을 더하여 새로운 '총 쓰레기(톤)' 컬럼 생성
df['총 쓰레기(톤)'] = df['일반생활(톤)'] + df['음식물류(톤)']

# 3. 결과 확인 (선택 사항)
print(df.head())

        년월   구역  일반생활(톤)  음식물류(톤)  재활용품(톤)  총 쓰레기(톤)
0  2019-01  1구역      524      427      NaN       951
1  2019-01  2구역      591      484      NaN      1075
2  2019-01  3구역      552      500      NaN      1052
3  2019-01  4구역      432      368      NaN       800
4  2019-01  5구역      601      544      NaN      1145


In [25]:
df = df.drop(columns=['일반생활(톤)', '음식물류(톤)','재활용품(톤)'])
print(df.head())

        년월   구역  총 쓰레기(톤)
0  2019-01  1구역       951
1  2019-01  2구역      1075
2  2019-01  3구역      1052
3  2019-01  4구역       800
4  2019-01  5구역      1145


In [27]:
df = df.drop(columns=['구역'])
print(df)

          년월  총 쓰레기(톤)
0    2019-01       951
1    2019-01      1075
2    2019-01      1052
3    2019-01       800
4    2019-01      1145
..       ...       ...
343  2023-10      1156
344  2023-10      1049
345  2023-10       870
346  2023-10      1219
347  2023-10       499

[348 rows x 2 columns]


In [40]:
df_grouped = df.groupby('년월')['총 쓰레기(톤)'].sum().reset_index()
print("년월별 총 쓰레기(톤) 합계:\n", df_grouped)

년월별 총 쓰레기(톤) 합계:
          년월  총 쓰레기(톤)
0   2019-01      5479
1   2019-02      5023
2   2019-03      4961
3   2019-04      5472
4   2019-05      5444
5   2019-06      5232
6   2019-07      6024
7   2019-08      5632
8   2019-09      5469
9   2019-10      5473
10  2019-11      5750
11  2019-12      5519
12  2020-01      5486
13  2020-02      5283
14  2020-03      5516
15  2020-04      5074
16  2020-05      5710
17  2020-06      6000
18  2020-07      5828
19  2020-08      5899
20  2020-09      5595
21  2020-10      5708
22  2020-11      5735
23  2020-12      5784
24  2021-01      5189
25  2021-02      4933
26  2021-03      5565
27  2021-04      5199
28  2021-05      5383
29  2021-06      5626
30  2021-07      5896
31  2021-08      5785
32  2021-09      5573
33  2021-10      5443
34  2021-11      5995
35  2021-12      5524
36  2022-01      5263
37  2022-02      5092
38  2022-03      5498
39  2022-04      5296
40  2022-05      5568
41  2022-06      5558
42  2022-07      5912
43  2022-08   

In [41]:
file_path = '쓰레기 배출/서울특별시 양천구_구역별쓰레기배출량_20231115.csv'
try:
    df_grouped.to_csv(file_path, index=False, encoding='cp949') # 또는 encoding='cp949'
    print(f"\n'{file_path}'에 변경 사항을 저장했습니다.")
except Exception as e:
    print(f"파일을 저장하는 동안 오류가 발생했습니다: {e}")



'쓰레기 배출/서울특별시 양천구_구역별쓰레기배출량_20231115.csv'에 변경 사항을 저장했습니다.
